In [66]:
import numpy as np
from numpy import linalg as LA
import math as m
import os
from matplotlib.image import imread
import matplotlib.pyplot as plt
from matplotlib import rcParams # for changing default values
import scipy.io as sio
from scipy.optimize import minimize
import timeit
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.integrate import odeint
from torch.autograd import grad

# set all fontsizes to 12
rcParams["font.size"] = 12
rcParams["axes.axisbelow"] = True # make sure grid is behind plots

## Example 1: Sparse Identification of Nonlinear  Dynamics

Artificial data

In [67]:
#Example
#
# x=3*exp(2*t) => dot{x}=2x
#
#create data
m=10 #number of data points
t=np.linspace(0,1,m)
np.random.seed(0) 
x=3*np.exp( 2*t )
xdot=6*np.exp( 2*t )+0.1*np.random.rand(m) #add some noise

#now create Xdot
dim_x=1
Xdot=np.zeros((m,dim_x))
Xdot[:,0]=xdot

Define model library

In [68]:
#Define model library
dim_Theta=3 
Theta=np.zeros((m,dim_Theta))
for i in range(m):
    Theta[i,0]=1.
    Theta[i,1]=x[i]
    Theta[i,2]=x[i]*x[i] #nterms=3

Sequential least squares

In [69]:
#sequential least square
def sparsifyDynamics(Theta,dXdt,lamb,n):
    Xi = np.linalg.lstsq(Theta,dXdt,rcond=None)[0] # Initial guess: Least-squares
    
    for k in range(10):
        smallinds = np.abs(Xi) < lamb # Find small coefficients, here is the threshold
        Xi[smallinds] = 0                          # and threshold
        for ind in range(n):                       # n is state dimension
            biginds = smallinds[:,ind] == 0
            # Regress dynamics onto remaining terms to find sparse Xi
            Xi[biginds,ind] = np.linalg.lstsq(Theta[:,biginds],dXdt[:,ind],rcond=None)[0]
            
    return Xi

Solve problem

In [70]:
lamb=0.1 #chosen threshold
Xi=sparsifyDynamics(Theta,Xdot,lamb,dim_x)
print(Xi)
    

[[0.        ]
 [2.00462313]
 [0.        ]]


## Example 2: Sparse Identification of Nonlinear  Dynamics, Lorenz example

Create artificial data

In [71]:
# create data    
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate
## Simulate the Lorenz System

dt = 0.01
T = 50
t = np.arange(dt,T+dt,dt)
beta = 8/3
sigma = 10
rho = 28
n = 3

def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

np.random.seed(123)
x0 = (-8,8,27)

x = integrate.odeint(lorenz_deriv, x0, t,rtol=10**(-12),atol=10**(-12)*np.ones_like(x0))
## Compute Derivative
dx = np.zeros_like(x)
for j in range(len(t)):
    dx[j,:] = lorenz_deriv(x[j,:],0,sigma,beta,rho)

Help function for polynomials

In [72]:
from itertools import combinations_with_replacement

def polynomial_library(X, degree=3, include_bias=True, return_names=True):
    """
    Build a polynomial feature library (monomials) up to the given degree.
    Works for any number of inputs (columns in X) and any degree >= 0.

    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        Input data (each column is a variable x1..x_n).
    degree : int
        Maximum total degree of monomials to include.
    include_bias : bool
        If True, include the constant column (1).
    return_names : bool
        If True, also return a list of column names for interpretability.

    Returns
    -------
    Phi : array, shape (n_samples, n_terms)
        Polynomial feature matrix [1, x_i, x_i x_j, ..., up to 'degree'].
    names : list of str (optional)
        Names like ['1','x1','x2','x1^2','x1*x2', ...].
    """
    X = np.asarray(X)
    if X.ndim == 1:
        X = X[:, None]  # allow 1D input

    n_samples, n_features = X.shape
    cols = []
    names = []

    # Degree 0 (bias)
    if include_bias:
        cols.append(np.ones((n_samples, 1)))
        if return_names:
            names.append("1")

    # Degrees 1..degree
    for d in range(1, degree + 1):
        for idxs in combinations_with_replacement(range(n_features), d):
            # product of selected columns; idxs is nondecreasing => unique monomial
            col = np.prod(X[:, idxs], axis=1, dtype=float)[:, None]
            cols.append(col)
            if return_names:
                # Build a readable name like x1^2*x3
                # Count powers per variable in this combo
                powers = np.bincount(idxs, minlength=n_features)
                parts = []
                for j, p in enumerate(powers):
                    if p == 0: 
                        continue
                    parts.append(f"x{j+1}" if p == 1 else f"x{j+1}^{p}")
                names.append("*".join(parts))

    Phi = np.hstack(cols) if cols else np.empty((n_samples, 0))

    return (Phi, names) if return_names else Phi

Create model library and create $\Theta$ from data

In [73]:
#create Theta from data
Theta, theta_library = polynomial_library(x, degree=3, include_bias=True)
print(theta_library)
lamb = 0.025 # sparsification limit
Xi = sparsifyDynamics(Theta,dx,lamb,n)

#now a trick to get informative output
Xi_object=Xi.astype(object)
theta_library_column = np.array(theta_library).reshape(-1, 1)
print(np.hstack((Xi_object, theta_library_column)))


['1', 'x1', 'x2', 'x3', 'x1^2', 'x1*x2', 'x1*x3', 'x2^2', 'x2*x3', 'x3^2', 'x1^3', 'x1^2*x2', 'x1^2*x3', 'x1*x2^2', 'x1*x2*x3', 'x1*x3^2', 'x2^3', 'x2^2*x3', 'x2*x3^2', 'x3^3']
[[0.0 0.0 0.0 '1']
 [-10.000000000000004 28.000000000000032 0.0 'x1']
 [10.000000000000004 -0.9999999999999668 0.0 'x2']
 [0.0 0.0 -2.666666666666667 'x3']
 [0.0 0.0 0.0 'x1^2']
 [0.0 0.0 1.0 'x1*x2']
 [0.0 -1.0000000000000013 0.0 'x1*x3']
 [0.0 0.0 0.0 'x2^2']
 [0.0 0.0 0.0 'x2*x3']
 [0.0 0.0 0.0 'x3^2']
 [0.0 0.0 0.0 'x1^3']
 [0.0 0.0 0.0 'x1^2*x2']
 [0.0 0.0 0.0 'x1^2*x3']
 [0.0 0.0 0.0 'x1*x2^2']
 [0.0 0.0 0.0 'x1*x2*x3']
 [0.0 0.0 0.0 'x1*x3^2']
 [0.0 0.0 0.0 'x2^3']
 [0.0 0.0 0.0 'x2^2*x3']
 [0.0 0.0 0.0 'x2*x3^2']
 [0.0 0.0 0.0 'x3^3']]


## Symbolic regression PySR package

### Example 1

### Create artificial data

In [ ]:
#from pysr import *
import pysr as pysr

In [ ]:
X = 2 * np.random.randn(100, 5)
y = 2 * np.cos(X[:, 3]) + X[:, 0] ** 2 - 2

Solve problem

In [ ]:
model = pysr.PySRRegressor(binary_operators=["+", "-", "*", "/"],unary_operators=["cos","sin"])
model.fit(X, y)
print(model)
print(f'Model in latex: {model.latex()}')


Result: $x_{0} x_{0} + \cos{\left(x_{3} \right)} + \cos{\left(x_{3} \right)} - 1.22 - 0.817 - -0.00713 - -0.0257$

## Example 2

Create data

In [ ]:
X = 2 * np.random.randn(100, 5)
y = 1 / X[:, 0]

Solve problem

In [ ]:
model = pysr.PySRRegressor(
    binary_operators=["+", "*"],
    unary_operators=["inv(x) = 1/x"],
    extra_sympy_mappings={"inv": lambda x: 1/x},
)
model.fit(X, y)
print(model)
print(f'Model in latex: {model.latex()}')


## Example 3

Artificial data

In [ ]:

X = 2 * np.random.randn(100, 5)
y = 1 / X[:, [0, 1, 2]]


Solution

In [ ]:
X = 2 * np.random.randn(100, 5)
y = 1 / X[:, [0, 1, 2]]

model = pysr.PySRRegressor(
    binary_operators=["+", "*"],
    unary_operators=["inv(x) = 1/x"],
    extra_sympy_mappings={"inv": lambda x: 1/x},
)
model.fit(X, y)
print(f'Model in latex: {model.latex()}')

from matplotlib import pyplot as plt
plt.scatter(y[:, 0], model.predict(X)[:, 0])
plt.xlabel('Truth')
plt.ylabel('Prediction')
plt.show()

### Lorenz example

Create data

In [ ]:
# create data    
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate
## Simulate the Lorenz System

dt = 0.01
T = 50
t = np.arange(dt,T+dt,dt)
beta = 8/3
sigma = 10
rho = 28
n = 3

def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

np.random.seed(123)
x0 = (-8,8,27)

x = integrate.odeint(lorenz_deriv, x0, t,rtol=10**(-12),atol=10**(-12)*np.ones_like(x0))
## Compute Derivative
dx = np.zeros_like(x)
for j in range(len(t)):
    dx[j,:] = lorenz_deriv(x[j,:],0,sigma,beta,rho)



Solve problem

In [ ]:
   
model = pysr.PySRRegressor(
    binary_operators=["+", "*"],
    unary_operators=["inv(x) = 1/x"],
    extra_sympy_mappings={"inv": lambda x: 1/x},
)
model.fit(x, dx)
print(f"Best equation: {model.get_best()}")
print(f'in latex:  {model.latex()}')


$\left(x_{0} \left(-1.00\right) + x_{1}\right) 10.0$,  

$x_{0} \left(28.0 + x_{2} \left(-1.00\right)\right) - 1.00 x_{1}$, 

$x_{0} x_{1} - 2.67 x_{2}$